In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
fake=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
true=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")


In [ ]:
#checking how the data looks like
fake.head(10)
fake.tail(10)

In [ ]:
#checking different subjecys and their counts
fake["subject"].value_counts()

In [ ]:
true.head()

In [ ]:
true.subject.value_counts()

In [ ]:
#creating a categories for whether fake or true
fake["category"]=1
true["category"]=0

In [ ]:
fake.head()

In [ ]:
#joining the data the two data frame and reseting index
df=pd.concat([fake ,true]).reset_index(drop=True)

In [ ]:
# to check the data
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(10,5))



graph = sns.countplot(x="category", data=df)
plt.title("Count of Fake and True News")

#removing boundary
graph.spines["right"].set_visible(False)
graph.spines["top"].set_visible(False)
graph.spines["left"].set_visible(False)

#annoting bars with the counts  
for p in graph.patches:
        height = p.get_height()
        graph.text(p.get_x()+p.get_width()/2., height + 0.2,height ,ha="center",fontsize=12)



In [ ]:
#creating a count plot for subject column
fig = plt.figure(figsize=(10,5))



graph = sns.countplot(x="subject", data=df)
plt.title("Count of Subjecs")

#removing boundary
graph.spines["right"].set_visible(False)
graph.spines["top"].set_visible(False)
graph.spines["left"].set_visible(False)

#annoting bars with the counts  
for p in graph.patches:
        height = p.get_height()
        graph.text(p.get_x()+p.get_width()/2., height + 0.2,height ,ha="center",fontsize=12)

In [ ]:
#checking the missing values in each columns
df.isna().sum()*100/len(df)

In [ ]:
df.head()

In [ ]:
# checking if there is empty string in TEXT column
blanks = []

# index and text of the document
for index, text in df["text"].items():  
    if text.isspace():  # if the string is only whitespace
        blanks.append(index)  # note the index

len(blanks)


In [ ]:
blanks


In [ ]:
#instead of dropping these values we are going to merge title with text

df["text"] =df["title"]+df["text"]

#we only need two columns rest can be ignored

df=df[["text","category"]]

In [ ]:

#importing libraries for cleaning puprose

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import spacy
import re

In [ ]:
#loading spacy library
nlp=spacy.load("en_core_web_sm")

#creating instance
lemma=WordNetLemmatizer()

In [ ]:
#creating list of stopwords containing stopwords from spacy and nltk

#stopwords of spacy
list1=nlp.Defaults.stop_words
print(len(list1))

#stopwords of NLTK
list2=stopwords.words('english')
print(len(list2))

#combining the stopword list
Stopwords=set((set(list1)|set(list2)))
print(len(Stopwords))

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize stopwords and lemmatizer
Stopwords = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

def clean_text(text):
    """
    Cleans input text by:
    - Converting to lowercase
    - Expanding contractions
    - Removing special characters
    - Removing stopwords
    - Lemmatizing words
    """
    text = text.lower()

    # Expanding common contractions
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)

    # Removing special characters and extra whitespace
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    # Lemmatization and stopword removal
    cleaned = []
    for word in text.split():
        if word not in Stopwords:
            cleaned.append(lemma.lemmatize(word))

    return " ".join(cleaned)


In [ ]:
df['clean_text'] = df['text'].apply(clean_text)


In [ ]:
from wordcloud import WordCloud

In [ ]:
#True News
plt.figure(figsize = (20,20))
Wc = WordCloud(max_words = 500 , width = 1600 , height = 800).generate(" ".join(df[df.category == 0].text))
plt.axis("off")
plt.imshow(Wc , interpolation = 'bilinear')

In [ ]:
#creating more intiuive wordcloud 

#pil is pillow and used for image manupulation
from PIL import Image

In [ ]:
#creating a mask of thumb
thumb="../input/images-coud/thumbs-up.png"
icon=Image.open(thumb)
mask=Image.new(mode="RGB",size=icon.size, color=(255,255,255))
mask.paste(icon, box=icon)

rgb_array=np.array(mask)


In [ ]:
#True News
plt.figure(figsize = (10,10))
Wc = WordCloud(mask=rgb_array,max_words = 2000 , width = 1600 ,
               height = 800)

Wc.generate(" ".join(df[df.category == 0].text))
plt.axis("off")
plt.imshow(Wc , interpolation = 'bilinear')


In [ ]:
#creating word cloud using skull image for fake news which depict that 
#fake news are dangerous 

skull="../input/images-coud/skull-icon.png"
icon=Image.open(skull)
mask=Image.new(mode="RGB",size=icon.size, color=(255,255,255))
mask.paste(icon, box=icon)

rgb_array=np.array(mask)

In [ ]:
#Fake News
plt.figure(figsize = (15,15))
Wc = WordCloud(mask=rgb_array,max_words = 2000 , width = 1600 ,
               height = 800)

Wc.generate(" ".join(df[df.category == 1].text))
plt.axis("off")
plt.imshow(Wc , interpolation = 'bilinear')

In [ ]:
#splitting the 
from sklearn.model_selection import train_test_split


X=df["text"] #feature 
y=df["category"] # traget

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#importing libraries to build a pipline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [ ]:
#this pipe line will take the text and vectorise it , and then TF-IDF, then fitting the model

text_clf=Pipeline([("tfidf",TfidfVectorizer()),("clf",LinearSVC())])
text_clf.fit(X_train,y_train)

In [ ]:
#making prediction using the model
predictions=text_clf.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test,predictions))

In [ ]:
#overall acuracy
print(metrics.accuracy_score(y_test,predictions))

In [ ]:
#confusion matrix
print(metrics.confusion_matrix(y_test,predictions))

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'



In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from transformers import pipeline, set_seed


In [ ]:
# Load the pre-trained GPT-2 model for text generation
generator = pipeline("text-generation", model="gpt2")


In [ ]:
def generate_fake_news(prompt, max_length=100):
    set_seed(42)  # ensures reproducible results
    result = generator(prompt, max_length=max_length, num_return_sequences=1)
    return result[0]['generated_text']


In [ ]:
prompt = "Breaking News: Scientists discover a portal to"
generated_news = generate_fake_news(prompt, max_length=120)

print("📰 Generated Fake News:\n")
print(generated_news)


In [ ]:
prompt = input("Enter a prompt for fake news: ")
print(generate_fake_news(prompt, max_length=100))


In [ ]:
with open("generated_fake_news.txt", "w") as f:
    f.write(generated_news)

print("✅ Article saved to 'generated_fake_news.txt'")


In [ ]:

# 🔁 Combined: Generate Fake News and Detect it
prompt = "Breaking News: A mysterious signal was received from Mars."

# Generate fake article
generated_article = generate_fake_news(prompt, max_length=120)
print("📰 Generated Article:")
print(generated_article)

# Detect using the trained model
label, confidence = detect_news(generated_article)
print("\n🔍 Detector Prediction:")
print(f"→ This article is: {label} ({confidence}% confidence)")
